# In Class 5 - Dario Placencio.ipynb

Consider data set "WorkersComp.csv".

In [2]:
# Import the required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


import nltk
# normalization
import unicodedata
# stopwords
from nltk.corpus import stopwords
# punctuation
import string
# clean using regex expression
import re
#stemming
from nltk.stem.snowball import SnowballStemmer
# lemmatize
from nltk.stem import WordNetLemmatizer

In [3]:
# nltk.download('stopwords')
sw = set(stopwords.words("english"))

In [4]:
# define stemmer
stemmer = SnowballStemmer("english")

In [5]:
# Read the data
dat = pd.read_csv("WorkersComp.csv")

In [6]:
# Print the first 5 rows
dat.head()

,ClaimNumber,DateTimeOfAccident,DateReported,Age,Gender,MaritalStatus,DependentChildren,DependentsOther,WeeklyWages,PartTimeFullTime,HoursWorkedPerWeek,DaysWorkedPerWeek,ClaimDescription,InitialIncurredCalimsCost,UltimateIncurredClaimCost
0,WC8285054,2002-04-09T07:00:00Z,2002-07-05T00:00:00Z,48,M,M,0,0,500.00,F,38.0,5,LIFTING TYRE INJURY TO RIGHT ARM AND WRIST INJURY,1500,4748.203388
1,WC6982224,1999-01-07T11:00:00Z,1999-01-20T00:00:00Z,43,F,M,0,0,509.34,F,37.5,5,STEPPED AROUND CRATES AND TRUCK TRAY FRACTURE ...,5500,6326.285819
2,WC5481426,1996-03-25T00:00:00Z,1996-04-14T00:00:00Z,30,M,U,0,0,709.10,F,38.0,5,CUT ON SHARP EDGE CUT LEFT THUMB,1700,2293.949087
3,WC9775968,2005-06-22T13:00:00Z,2005-07-22T00:00:00Z,41,M,S,0,0,555.46,F,38.0,5,DIGGING LOWER BACK LOWER BACK STRAIN,15000,17786.487170
4,WC2634037,1990-08-29T08:00:00Z,1990-09-27T00:00:00Z,36,M,M,0,0,377.10,F,38.0,5,REACHING ABOVE SHOULDER LEVEL ACUTE MUSCLE STR...,2800,4014.002925


1) Build a unigram (i.e. remove stop words, stemming, Lemmatization). Report the result for the first 3 claims.

Removing Stopword

In [7]:
# Remove stop words from Claim Description  
dat['ClaimDescription'] = dat['ClaimDescription'].apply(lambda x: ' '.join([word for word in x.split() if word not in (sw)]))

Stemming

In [8]:
Stem = []
for Claim in dat.ClaimDescription:
    text = Claim.lower() # lowerlize
    text = re.sub(r'[^a-zA-Z0-9]', ' ', text)
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    text = text.split()  # plit
    tokens=[stemmer.stem(i) for i in text if (i not in string.punctuation)&(i not in sw)]
    Stem.append(tokens)

In [9]:
dat['Stem'] = Stem

In [10]:
Stem[0:5]

[['lift', 'tyre', 'injuri', 'right', 'arm', 'wrist', 'injuri'],
 ['step', 'around', 'crate', 'truck', 'tray', 'fractur', 'left', 'forearm'],
 ['cut', 'sharp', 'edg', 'cut', 'left', 'thumb'],
 ['dig', 'lower', 'back', 'lower', 'back', 'strain'],
 ['reach',
  'shoulder',
  'level',
  'acut',
  'muscl',
  'strain',
  'left',
  'side',
  'stomach']]

Lemmatization

In [11]:
#nltk.download('wordnet')
#nltk.download('omw-1.4')
# define lemmatizer
lemmatizer = WordNetLemmatizer()

In [12]:
Lemm = []
lemmatizer = WordNetLemmatizer()
for Claim in dat.Stem:
    #text = Claim.lower()
    #text = re.sub(r'[^a-zA-Z0-9]', ' ', text)
    #text = re.sub(r'https?://\S+|www\.\S+', '', text)
    #text = text.split()
    tokens=[lemmatizer.lemmatize(i) for i in Claim] #if (i not in string.punctuation)&(i not in sw)]
    Lemm.append(tokens)

In [13]:
Lemm[0:5]

[['lift', 'tyre', 'injuri', 'right', 'arm', 'wrist', 'injuri'],
 ['step', 'around', 'crate', 'truck', 'tray', 'fractur', 'left', 'forearm'],
 ['cut', 'sharp', 'edg', 'cut', 'left', 'thumb'],
 ['dig', 'lower', 'back', 'lower', 'back', 'strain'],
 ['reach',
  'shoulder',
  'level',
  'acut',
  'muscl',
  'strain',
  'left',
  'side',
  'stomach']]

Unigram

In [14]:
### N-grams

from nltk import ngrams

In [15]:
# define function for uni-gram
def uni(tokens):
    return tokens

dat['Stem_uni'] = dat['Stem'].apply(lambda x: uni(x)) # uni-gram

In [16]:
# Preview the first 5 rows of the Stem_uni column
dat['Stem_uni'].head()

0      [lift, tyre, injuri, right, arm, wrist, injuri]
1    [step, around, crate, truck, tray, fractur, le...
2                  [cut, sharp, edg, cut, left, thumb]
3              [dig, lower, back, lower, back, strain]
4    [reach, shoulder, level, acut, muscl, strain, ...
Name: Stem_uni, dtype: object

2) Create the term-document matrix in the bag-of-words model. Report the dimension of this matrix. Show the first term frequency for the first 3 claims.

In [31]:
# Create the term document matrix in the bag-of-words model, do not use TD-IDF, use the count of the words
from sklearn.feature_extraction.text import CountVectorizer

# define the list of documents
docs = dat['Stem_uni']

# Create an instance of CountVectorizer
vectorizer = CountVectorizer()

# define the vocabulary
vocab = set([word for tokens in docs for word in tokens])

# Use fit_transform() to create the term-document matrix
term_doc_matrix = vectorizer.fit_transform(vocab)

# Convert the term-document matrix to an array for easier manipulation
term_doc_matrix = term_doc_matrix.toarray()

# Get the feature names (i.e., the words)
feature_names = vectorizer.get_feature_names()

c:\Users\Placenciohid\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [33]:
# Report Matrix Dimensions
print("Matrix Dimensions:")
print(term_doc_matrix.shape)

Matrix Dimensions:
(2721, 2721)


In [35]:
# Create a dataframe from the term-document matrix
df = pd.DataFrame(term_doc_matrix, columns=feature_names)

In [36]:
# Preview the first 3 rows of the dataframe
df.head(3)

,abalon,abcess,abdomin,abil,abnorm,abras,abscess,abus,accept,access,...,wrestl,wrist,write,wrong,yacht,yard,yarn,yoghurt,zinc,zipper
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


3) Calcuate TF-IDF scores using unigram. Report the dimension of the TF-IDF matrix. Show the TF-IDF scores for the first 3 claims.

In [17]:
### TFIDF

from sklearn.feature_extraction.text import TfidfVectorizer

In [18]:
comm_clean=[]
for l in Lemm:
    if l:
        comm=" ".join(l)
        comm_clean.append(comm)
comm_clean[0:5]

['lift tyre injuri right arm wrist injuri',
 'step around crate truck tray fractur left forearm',
 'cut sharp edg cut left thumb',
 'dig lower back lower back strain',
 'reach shoulder level acut muscl strain left side stomach']

In [19]:
tfidf = TfidfVectorizer(ngram_range=(1,1),stop_words = 'english')
TFIDF = tfidf.fit_transform(comm_clean)

In [20]:
# use toarray() to create dense matrix from sparse matrix
tfidf_mat = pd.DataFrame(TFIDF.toarray(), columns=tfidf.get_feature_names())

c:\Users\Placenciohid\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [21]:
# Show the term frequency matrix for the first 3 claims
tfidf_mat.head(3)

,abalon,abcess,abdomin,abil,abnorm,abras,abscess,abus,accept,access,...,wrestl,wrist,write,wrong,yacht,yard,yarn,yoghurt,zinc,zipper
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.33688,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
# Report Matrix Dimensions
tfidf_mat.shape

(52978, 2674)